In [1]:
%load_ext autoreload
%autoreload 2

import sys; 
sys.path.insert(0, '../../esinet')
sys.path.insert(0, '../')

import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import pearsonr
import mne
from esinet import Simulation
from esinet.forward import get_info, create_forward_model
from esinet.util import unpack_fwd
from invert.cmaps import parula
pp = dict(surface='white', hemi='both')

# Get Forward Model

In [2]:
info = get_info(kind='biosemi128')
fwd = create_forward_model(info=info, sampling='ico3')

leadfield, pos = unpack_fwd(fwd)[1:3]
n_chans, n_dipoles = leadfield.shape

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    1.9s remaining:    3.2s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    2.0s remaining:    1.2s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    2.2s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.1s finished


NameError: name 'cdist' is not defined

# Get sample data

In [20]:
# settings = dict(number_of_sources=1, extents=40, duration_of_trial=0.01, target_snr=99999999999)
settings = dict(number_of_sources=4, extents=(1, 40), duration_of_trial=1, target_snr=99999)

sim = Simulation(fwd, info, settings).simulate(2)
stc = sim.source_data[0]
evoked = sim.eeg_data[0].average()
M = evoked.data

brain = stc.plot(**pp)
brain.add_text(0.1, 0.9, 'Ground Truth', 'title',
               font_size=14)

Using control points [1.24460026e-10 3.62813642e-09 4.35199337e-08]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


Using control points [7.38455564e-10 1.73060568e-09 8.91730342e-09]
Using control points [7.93994731e-10 9.41549211e-10 4.44043846e-09]


In [21]:
from invert import make_inverse_operator, apply_inverse_operator
solver = "bbmf"
inverse_operator = make_inverse_operator(fwd, solver=solver, evoked=evoked)
stc_hat = apply_inverse_operator(evoked, inverse_operator, fwd)

# stc.plot(**pp, brain_kwargs=dict(title="Ground Truth"))
stc_hat.plot(**pp, brain_kwargs=dict(title=solver))


Using 4 temporal mode(s)
ReML Iteration 0: -59.13185734020146
Free-energy:  [[nan nan]
 [nan nan]]
final h:  [ 8.50707424 -8.82294525]
Using control points [9.65256134e-10 1.59565160e-09 6.72217650e-08]


c:\Users\lukas\Dokumente\projects\invert\dev\..\invert\multiple_sparse_priors.py:678: RuntimeWarning: divide by zero encountered in true_divide
  # complexity - KL(Ph,hP)
c:\Users\lukas\Dokumente\projects\invert\dev\..\invert\multiple_sparse_priors.py:678: RuntimeWarning: invalid value encountered in log
  # complexity - KL(Ph,hP)


For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


Using control points [7.38455564e-10 1.73060568e-09 8.91730342e-09]
Using control points [3.14774639e-10 4.81723080e-10 2.16193971e-08]
Using control points [7.93994731e-10 9.41549211e-10 4.44043846e-09]


In [ ]:
from invert import all_solvers, make_inverse_operator, apply_inverse_operator

stc.plot(**pp, brain_kwargs=dict(title="Ground Truth"))

for solver in all_solvers:
    inverse_operator = make_inverse_operator(fwd, solver=solver)
    stc_hat = apply_inverse_operator(evoked, inverse_operator, fwd)
    stc_hat.plot(**pp, brain_kwargs=dict(title=solver))


Using control points [1.82213031e-09 5.54186393e-09 3.54210799e-08]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`
Using control points [1.76855722e-09 2.36892900e-09 1.18219634e-08]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`
Using control points [1.87071326e-09 2.48858259e-09 1.10969785e-08]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`
Using control points [3.06619344e-07 4.17032838e-07 2.49949036e-06]
For automatic theme detection, "darkdetect" has to be installed! You can install

: 

: 